In [ ]:
# default_exp data

# data.py

> modules for inspecting the data

In [ ]:
#hide
from nbdev.showdoc import *
from fastai.vision import *

In [ ]:
data_path = Path('data')
data_path.ls()

[PosixPath('data/train_2d.pth.tar'),
 PosixPath('data/train_3d.pth.tar'),
 PosixPath('data/test_2d.pth.tar'),
 PosixPath('data/test_2d_ft.pth.tar'),
 PosixPath('data/train_2d_ft.pth.tar'),
 PosixPath('data/test_3d.pth.tar'),
 PosixPath('data/stat_3d.pth.tar')]

In [ ]:
stat_3d = torch.load(data_path/'stat_3d.pth.tar')

In [ ]:
stat_3d.keys()

dict_keys(['mean', 'std', 'dim_use', 'dim_ignore', 'train_root', 'test_root'])

In [ ]:
stat_3d['mean']

array([   0.      ,    0.      ,    0.      ,   -0.255653, ...,  -36.834286,    0.524591, -165.867774,  -36.834286])

In [ ]:
train_set_3d = torch.load(data_path/'train_3d.pth.tar')

In [ ]:
train_set_3d[(1, 'Directions', 'Directions 1.54138969.h5')]

array([[ 1.120139,  0.824709,  0.664921,  0.168664, ...,  0.462295,  0.777653,  0.875328,  0.997535],
       [ 1.119314,  0.822044,  0.668415,  0.167789, ...,  0.461423,  0.768301,  0.870435,  0.999916],
       [ 1.118593,  0.820576,  0.671385,  0.166983, ...,  0.459609,  0.762353,  0.867354,  0.999718],
       [ 1.118141,  0.821497,  0.673104,  0.165622, ...,  0.457249,  0.759257,  0.865367,  0.997495],
       ...,
       [ 0.974209, -0.650475,  1.048559,  0.719905, ...,  0.477995,  0.84735 ,  0.936249,  0.917143],
       [ 0.973033, -0.650405,  1.051138,  0.719954, ...,  0.477686,  0.834714,  0.93506 ,  0.919616],
       [ 0.971866, -0.650319,  1.053692,  0.719993, ...,  0.477847,  0.819343,  0.933946,  0.922678],
       [ 0.970378, -0.651391,  1.056826,  0.718762, ...,  0.477228,  0.802321,  0.934879,  0.921835]])

In [ ]:
train_set_2d = torch.load(data_path/'train_2d.pth.tar')

In [ ]:
train_set_2d[(1, 'Directions', 'Directions 1.54138969.h5')]

array([[-0.542063,  0.397749, -0.258918,  0.464261, ..., -0.090757,  0.504997, -0.130928,  0.843619],
       [-0.542026,  0.39791 , -0.259014,  0.464274, ..., -0.089592,  0.504665, -0.134512,  0.841068],
       [-0.541974,  0.398126, -0.259075,  0.464425, ..., -0.088175,  0.504291, -0.13675 ,  0.839494],
       [-0.541874,  0.398428, -0.259037,  0.464823, ..., -0.086743,  0.503481, -0.137833,  0.838531],
       ...,
       [-0.215286,  0.256225, -0.000856,  0.222175, ...,  0.166997,  0.389731,  0.146148,  0.775821],
       [-0.215104,  0.25639 , -0.000931,  0.222386, ...,  0.166005,  0.390665,  0.141417,  0.775354],
       [-0.214887,  0.256434, -0.000975,  0.222479, ...,  0.165197,  0.3919  ,  0.135659,  0.774865],
       [-0.214759,  0.256285, -0.001163,  0.22231 , ...,  0.163877,  0.392904,  0.129249,  0.77524 ]])

## list of actions

In [ ]:
actions = ["Directions",]
#            "Discussion",
#            "Eating",
#            "Greeting",
#            "Phoning",
#            "Photo",
#            "Posing",
#            "Purchases",
#            "Sitting",
#            "SittingDown",
#            "Smoking",
#            "Waiting",
#            "WalkDog",
#            "Walking",
#            "WalkTogether"]

In [ ]:
len(actions)

1

In [ ]:
stat_3d = torch.load(data_path/'stat_3d.pth.tar')
type(stat_3d)

<class 'dict'>

In [ ]:
import os
import torch
from torch.utils.data import Dataset


TRAIN_SUBJECTS = [1, 5, 6, 7, 8]
TEST_SUBJECTS = [9, 11]


class Human36M(Dataset):
    def __init__(self, actions, data_path, use_hg=True, is_train=True):
        """
        :param actions: list of actions to use
        :param data_path: path to dataset
        :param use_hg: use stacked hourglass detections
        :param is_train: load train/test dataset
        """

        self.actions = actions
        self.data_path = data_path

        self.is_train = is_train
        self.use_hg = use_hg

        self.train_inp, self.train_out, self.test_inp, self.test_out = [], [], [], []
        self.train_meta, self.test_meta = [], []

        # loading data
        if self.use_hg:
            train_2d_file = 'train_2d_ft.pth.tar'
            test_2d_file = 'test_2d_ft.pth.tar'
        else:
            train_2d_file = 'train_2d.pth.tar'
            test_2d_file = 'test_2d.pth.tar'

        if self.is_train:
            # load train data
            self.train_3d = torch.load(os.path.join(data_path, 'train_3d.pth.tar'))
            self.train_2d = torch.load(os.path.join(data_path, train_2d_file))
            for k2d in self.train_2d.keys():
                (sub, act, fname) = k2d
                k3d = k2d
                k3d = (sub, act, fname[:-3]) if fname.endswith('-sh') else k3d
                num_f, _ = self.train_2d[k2d].shape
                assert self.train_3d[k3d].shape[0] == self.train_2d[k2d].shape[0], '(training) 3d & 2d shape not matched'
                for i in range(num_f):
                    self.train_inp.append(self.train_2d[k2d][i])
                    self.train_out.append(self.train_3d[k3d][i])

        else:
            # load test data
            self.test_3d = torch.load(os.path.join(data_path, 'test_3d.pth.tar'))
            self.test_2d = torch.load(os.path.join(data_path, test_2d_file))
            for k2d in self.test_2d.keys():
                (sub, act, fname) = k2d
                if act not in self.actions:
                    continue
                k3d = k2d
                k3d = (sub, act, fname[:-3]) if fname.endswith('-sh') else k3d
                num_f, _ = self.test_2d[k2d].shape
                assert self.test_2d[k2d].shape[0] == self.test_3d[k3d].shape[0], '(test) 3d & 2d shape not matched'
                for i in range(num_f):
                    self.test_inp.append(self.test_2d[k2d][i])
                    self.test_out.append(self.test_3d[k3d][i])

    def __getitem__(self, index):
        if self.is_train:
            inputs = torch.from_numpy(self.train_inp[index]).float()
            outputs = torch.from_numpy(self.train_out[index]).float()

        else:
            inputs = torch.from_numpy(self.test_inp[index]).float()
            outputs = torch.from_numpy(self.test_out[index]).float()

        return inputs, outputs

    def __len__(self):
        if self.is_train:
            return len(self.train_inp)
        else:
            return len(self.test_inp)


In [ ]:
dataset = Human36M(actions, data_path, False, True)
type(dataset)

<class '__main__.Human36M'>

In [ ]:
test_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=False, num_workers=8)

In [ ]:
x, y = next(iter(test_loader))

In [ ]:
y = y[0]
x = x[0]

In [ ]:
def unNormalizeData(normalized_data, data_mean, data_std, dimensions_to_use):
    T = normalized_data.shape[0]  # Batch size
    D = data_mean.shape[0]  # 96

    orig_data = np.zeros((T, D), dtype=np.float32)

    orig_data[:, dimensions_to_use] = normalized_data

    # Multiply times stdev and add the mean
    stdMat = data_std.reshape((1, D))
    stdMat = np.repeat(stdMat, T, axis=0)
    meanMat = data_mean.reshape((1, D))
    meanMat = np.repeat(meanMat, T, axis=0)
    orig_data = np.multiply(orig_data, stdMat) + meanMat
    return orig_data

In [ ]:
stat_3d.keys()

dict_keys(['mean', 'std', 'dim_use', 'dim_ignore', 'train_root', 'test_root'])

In [ ]:
len(stat_3d['std'])

96

In [ ]:
len(y)

48

In [ ]:
stat_3d['dim_use']

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
       46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 75, 76, 77, 78, 79, 80, 81, 82, 83])

In [ ]:
stat_3d['dim_ignore']

array([ 0,  1,  2, 12, 13, 14, 15, 16, 17, 27, 28, 29, 30, 31, 32, 33, 34, 35, 48, 49, 50, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])